In [34]:
%matplotlib inline

import os

import pandas as pd
import seaborn as sns
import numpy as np
import scipy.stats as stats

import matplotlib as mpl
import matplotlib.pyplot as plt

import cv2

from PIL import Image

import scipy.ndimage as ndimage 
from scipy.interpolate import RectBivariateSpline 

In [35]:
directory = 'test_images'
filename = 'test1.bmp'
f = os.path.join(directory, filename)  
bacteria = cv2.imread(f)
image = cv2.cvtColor(bacteria, cv2.COLOR_BGR2RGB)

filename = 'test1_jpg.jpg'
test1_jpg = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
f = os.path.join(directory, filename)  
cv2.imwrite(f, test1_jpg)

filename = 'test1_jpg.jpg'
f = os.path.join(directory, filename)  
image = Image.open(f) 
t1j = np.array(image) 
t1j0 = t1j[:,:,0]
filename = 'test1_jpg_channel0.jpg'
f = os.path.join(directory, filename)  
cv2.imwrite(f, t1j0)
image = Image.open(f) 
i_in = np.array(image) 


In [36]:
print(i_in.shape)

(125, 166)


In [37]:
psf=4
gain=1
background=0
temporal='mean'
options = {
        'gain': gain,
        'background': background,  
        'temporal': temporal 
    }

In [38]:
gain = options['gain']
window_radius = int(np.ceil(options['background']))
psf = psf / 1.6651  

# Апскейл
number_row_initial, number_column_initial = i_in.shape
x0 = np.linspace(-0.5, 0.5, number_column_initial)
y0 = np.linspace(-0.5, 0.5, number_row_initial)

x = np.linspace(-0.5, 0.5, round(5 * number_column_initial / psf)) 
y = np.linspace(-0.5, 0.5, round(5 * number_row_initial / psf)) 

# Ядра Собеля для вычисления градиентов
Sobel_x = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
Sobel_y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

single_frame_i_in = i_in 
local_minimum = np.zeros((number_row_initial, number_column_initial)) 
single_frame_i_in_localmin = np.zeros((number_row_initial, number_column_initial))

In [39]:
for u in range(number_row_initial): 
    for v in range(number_column_initial): 
        sub_window = single_frame_i_in[max(0, u - window_radius):min(number_row_initial, u + window_radius + 1),
                        max(0, v - window_radius):min(number_column_initial, v + window_radius + 1)] # Вырезаем окно вокруг текущего пикселя
        local_minimum[u, v] = np.min(sub_window) # Находим минимальное значение пикселя в окне
        single_frame_i_in_localmin[u, v] = single_frame_i_in[u, v] - local_minimum[u, v] # Вычитаем локальный минимум из исходного изображения

In [40]:
# Апскейл
interp_localmin = RectBivariateSpline(y0, x0, single_frame_i_in_localmin) 
single_frame_localmin_magnified = interp_localmin(y, x) 
single_frame_localmin_magnified[single_frame_localmin_magnified < 0] = 0
single_frame_localmin_magnified = np.pad(single_frame_localmin_magnified, 10, mode='constant')

interp_i = RectBivariateSpline(y0, x0, single_frame_i_in) 
single_frame_i_magnified = interp_i(y, x) 
single_frame_i_magnified[single_frame_i_magnified < 0] = 0 
single_frame_i_magnified = np.pad(single_frame_i_magnified, 10, mode='constant') # Паддинг

number_row, number_column = single_frame_i_magnified.shape

In [41]:
# Нормализуем изображение
i_normalized = single_frame_localmin_magnified / (
        ndimage.gaussian_filter(single_frame_localmin_magnified, 10) + 1e-5)

# Вычисляем градиенты с помощью свертки с ядром Собеля
gradient_y = ndimage.convolve(i_normalized, Sobel_x, mode='reflect')
gradient_x = ndimage.convolve(i_normalized, Sobel_y, mode='reflect')
# Нормализуем градиенты
gradient_x = gradient_x / (i_normalized + 1e-5)
gradient_y = gradient_y / (i_normalized + 1e-5)

In [42]:
# Вычисляем смещение пикселей на основе градиента
gain_value = 0.5 * gain + 1 
displacement_x = gain_value * gradient_x
displacement_y = gain_value * gradient_y
displacement_x[np.abs(displacement_x) > 10] = 0
displacement_y[np.abs(displacement_y) > 10] = 0 

In [43]:
single_frame_i_out = np.zeros((number_row, number_column))
for nx in range(10, number_row - 10): # по строкам масштабированного изображения
    for ny in range(10, number_column - 10): # по столбцам масштабированного изображения
        weighted1 = (1 - abs(displacement_x[nx, ny] - int(displacement_x[nx, ny]))) * (
                1 - abs(displacement_y[nx, ny] - int(displacement_y[nx, ny])))
        # Вычисляем веса для четырех соседних пикселей на основе дробной части смещений
        weighted2 = (1 - abs(displacement_x[nx, ny] - int(displacement_x[nx, ny]))) * abs(
            displacement_y[nx, ny] - int(displacement_y[nx, ny]))
        weighted3 = abs(displacement_x[nx, ny] - int(displacement_x[nx, ny])) * (
                1 - abs(displacement_y[nx, ny] - int(displacement_y[nx, ny])))
        weighted4 = abs(displacement_x[nx, ny] - int(displacement_x[nx, ny])) * abs(
            displacement_y[nx, ny] - int(displacement_y[nx, ny]))

        coordinate1 = [int(displacement_x[nx, ny]), int(displacement_y[nx, ny])]
        coordinate2 = [int(displacement_x[nx, ny]),
                        int(displacement_y[nx, ny]) + int(np.sign(displacement_y[nx, ny]))]
        coordinate3 = [int(displacement_x[nx, ny]) + int(np.sign(displacement_x[nx, ny])),
                        int(displacement_y[nx, ny])]
        coordinate4 = [int(displacement_x[nx, ny]) + int(np.sign(displacement_x[nx, ny])),
                        int(displacement_y[nx, ny]) + int(np.sign(displacement_y[nx, ny]))]

        single_frame_i_out[nx + coordinate1[0], ny + coordinate1[1]] += weighted1 * single_frame_i_magnified[nx, ny]
        # Применяем взвешенное переназначение пикселей
        single_frame_i_out[nx + coordinate2[0], ny + coordinate2[1]] += weighted2 * single_frame_i_magnified[nx, ny]
        single_frame_i_out[nx + coordinate3[0], ny + coordinate3[1]] += weighted3 * single_frame_i_magnified[nx, ny]
        single_frame_i_out[nx + coordinate4[0], ny + coordinate4[1]] += weighted4 * single_frame_i_magnified[nx, ny]

In [44]:
#Удаляем паддинг
single_frame_i_out = single_frame_i_out[10:-10, 10:-10]
single_frame_i_magnified = single_frame_i_magnified[10:-10, 10:-10]

In [46]:
directory = 'test_images'
filename = 'test1_channel0_OUT.jpg'
r = single_frame_i_magnified
f = os.path.join(directory, filename)  
cv2.imwrite(f, r)

True